# Summary:

#### In this notebook, the trial runs from the preceding notebook ('01_So_What.ipynb') are systematically extended to try to find an optimal configuration for the two models. The model parameters to be fine tuned are: number of epochs, number of layers and the number of units.

#### The best performing model configuration will be used in the notebook '03_Blue_in_Green.ipynb' to analyze the impact of adding noise to the clean dataset.


# Table of contents
* [1. Load modules](#Part1_link)
* [2. Setup data](#Part2_link)
<br >&nbsp;&nbsp;&nbsp;[2.1  Generate data, separate testing and validation set and standardize testing data](#Part2.1_link)
* [3. Setup models and evaluate for various hyper-parameter choices](#Part3_link)
<br >&nbsp;&nbsp;&nbsp;[3.1 Compile and fit LSTM and RNN model](#Part3.1_link)
* [4. Visualize results](#Part4_link)

<a id='Part1_link'></a>
# 1. Load modules

In [1]:
import sys
sys.path.append("../src/")
import Kind_of_Blue  # own class with a collection of methods used in this analysis

import tensorflow as tf

import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

import numpy as np
import pandas as pd


<a id='Part2_link'></a>
# 2. Setup data

<a id='Part2.1_link'></a>
### 2.1 Generate data, separate testing and validation set and standardize testing data

The following steps are repeated from the previous notebook, '01_So_What.ipynb', and are grouped into one single step here for simplicity.

In [2]:
# set a range of dates on which the observations are made
idx = pd.date_range(end='7/1/2020', periods=5*364, freq='d')

# take a sine function as the observations
num_periods = 10  # number of sine periods
observations = [np.sin(2*np.pi*num_periods*x/len(idx)) for x in range(len(idx))]
print('number of observations in time series: {}'.format(len(observations)))

# initialize dataframe to store time series
df = pd.DataFrame(data=observations, columns=['observations'])
df.index = idx

# initialize object
mdq = Kind_of_Blue.Kind_of_Blue()

# load dataframe into object
mdq._selected_features = ['observations']
mdq.df = df

# train-validation split ratio as class attribute set to 70%
print('train split ratio = ', mdq.TRAIN_SPLIT_RATIO)

# initialize dataset from dataframe 
mdq.initialize_dataset()
print('loaded data set length: {}'.format(len(mdq._dataset)))

# standardize data
mdq.standardize_data()

# check that mean equals zero and the standard deviation is one
print('mean: {}, std: {}'.format(round(np.mean(mdq._dataset), 2), round(np.std(mdq._dataset), 2)))

# set number of time points for 1/ future forecasting points and 2/ the past, historical time points
future_target_size = int(365/52)
past_history_size = int(1*365)

# set batch size
batch_size = 32

# generate train and validation data
mdq.generate_train_and_val_data(future_target_size=future_target_size, past_history_size=past_history_size
                                , batch_size=batch_size)

print('number of training samples: {}'.format(mdq._num_samples))

number of observations in time series: 1820
train split ratio =  0.7
loaded data set length: 1820
mean: 0.0, std: 1.0
debug3: check what buffer_size actually does! and why is data shape always (..., ..., 1) <- 1???
training set shape: x:(909, 365, 1), y:(909, 7, 1)
validation set shape: x:(174, 365, 1), y:(174, 7, 1)
number of training samples: 909


<a id='Part3_link'></a>
# 3. Setup models and evaluate for various hyper-parameter choices

<a id='Part3.1_link'></a>
### 3.1 Compile and fit LSTM and RNN model

In [3]:
# generator for configurations to be iterated over

def config_generator():
    
    unit_choices = [256, 512]  # number of units in each neural network layer
    layer_choices = [2]  # total number of layers
    epoch_choices = [50, 100]  # number of epochs the model is trained on
    
    for units in unit_choices:
        for num_layers in layer_choices:
            for epochs in epoch_choices:
                yield units, num_layers, epochs


In [ ]:
# iterations over the model parameter configurations are done for both LSTM as well as RNN model
model_types = ['LSTM', 'RNN']

# set number of steps per epoch
num_samples = mdq._num_samples
steps_per_epoch = int(num_samples/batch_size)
validation_steps = int(steps_per_epoch/2)

# initialize results dictionary
res = {'model_type': [], 'epochs': [], 'num_layers': [], 'units': [], 'val_mse': []
       , 'mse': [], 'total_training_time': []}

for units, num_layers, epochs in config_generator():
    for model_type in model_types:

        print('currently running {} model'.format(model_type))

        # compile model
        mdq.compile_model(units=units, num_layers=num_layers, model_type=model_type)

        # fit model
        mdq.fit_model(epochs=epochs, steps_per_epoch=steps_per_epoch
                      ,validation_steps=validation_steps, model_type=model_type)
        
        # get errors
        history = mdq._histories[model_type]
        val_mse = history.history['val_mse'][-1]
        mse = history.history['mse'][-1]
        
        # get total training time
        total_training_time = sum(mdq._time_callbacks[model_type].times)
        
        # append results to results dictionary
        res['model_type'].append(model_type)
        res['epochs'].append(epochs)
        res['num_layers'].append(num_layers)
        res['units'].append(units)
        res['val_mse'].append(val_mse)
        res['mse'].append(mse)
        res['total_training_time'].append(total_training_time)


currently running LSTM model
debugLSTM: should an LSTM layer or a Dense layer be added?
Epoch 1/50
28/28 [==============================] - 37s 1s/step - loss: 0.2781 - mse: 0.2781 - val_loss: 0.0304 - val_mse: 0.0304
Epoch 2/50
28/28 [==============================] - 35s 1s/step - loss: 0.0358 - mse: 0.0358 - val_loss: 0.0437 - val_mse: 0.0437
Epoch 3/50
28/28 [==============================] - 34s 1s/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.0103 - val_mse: 0.0103
Epoch 4/50
28/28 [==============================] - 33s 1s/step - loss: 0.0193 - mse: 0.0193 - val_loss: 0.0069 - val_mse: 0.0069
Epoch 5/50
28/28 [==============================] - 37s 1s/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.0068 - val_mse: 0.0068
Epoch 6/50
28/28 [==============================] - 37s 1s/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 7/50
28/28 [==============================] - 35s 1s/step - loss: 0.0136 - mse: 0.0136 - val_loss: 0.0062 - val_mse: 0.0062
Ep

28/28 [==============================] - 4s 138ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0050 - val_mse: 0.0050
Epoch 12/50
28/28 [==============================] - 5s 170ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.0033 - val_mse: 0.0033
Epoch 13/50
28/28 [==============================] - 4s 144ms/step - loss: 3.1486 - mse: 3.1486 - val_loss: 0.7659 - val_mse: 0.7659
Epoch 14/50
28/28 [==============================] - 4s 136ms/step - loss: 2.0366 - mse: 2.0366 - val_loss: 1.7539 - val_mse: 1.7539
Epoch 15/50
28/28 [==============================] - 4s 138ms/step - loss: 0.6447 - mse: 0.6447 - val_loss: 0.1740 - val_mse: 0.1740
Epoch 16/50
28/28 [==============================] - 4s 145ms/step - loss: 0.2414 - mse: 0.2414 - val_loss: 0.1256 - val_mse: 0.1256
Epoch 17/50
28/28 [==============================] - 4s 136ms/step - loss: 0.2012 - mse: 0.2012 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 18/50
28/28 [==============================] - 8s 295ms/step - loss: 0.1909

28/28 [==============================] - 36s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 25/100
28/28 [==============================] - 36s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 26/100
28/28 [==============================] - 37s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 27/100
28/28 [==============================] - 37s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 28/100
28/28 [==============================] - 50s 2s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 29/100
28/28 [==============================] - 38s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 30/100
28/28 [==============================] - 35s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 31/100
28/28 [==============================] - 35s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 32/100
28/28 [==============================] -

28/28 [==============================] - 40s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 93/100
28/28 [==============================] - 33s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 94/100
28/28 [==============================] - 31s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 95/100
28/28 [==============================] - 33s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 96/100
28/28 [==============================] - 38s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 97/100
28/28 [==============================] - 34s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 98/100
28/28 [==============================] - 34s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 99/100
28/28 [==============================] - 34s 1s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 100/100
28/28 [==============================] 

Epoch 54/100
28/28 [==============================] - 4s 146ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 6.2522e-04 - val_mse: 6.2522e-04
Epoch 55/100
28/28 [==============================] - 4s 152ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0010 - val_mse: 0.0010
Epoch 56/100
28/28 [==============================] - 4s 149ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0092 - val_mse: 0.0092
Epoch 57/100
28/28 [==============================] - 4s 148ms/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.0124 - val_mse: 0.0124
Epoch 58/100
28/28 [==============================] - 4s 150ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.0027 - val_mse: 0.0027
Epoch 59/100
28/28 [==============================] - 4s 158ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 60/100
28/28 [==============================] - 4s 145ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0165 - val_mse: 0.0165
Epoch 61/100
28/28 [==============================] - 

28/28 [==============================] - 137s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 16/50
28/28 [==============================] - 130s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 17/50
28/28 [==============================] - 129s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 18/50
28/28 [==============================] - 135s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 19/50
28/28 [==============================] - 136s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 20/50
28/28 [==============================] - 129s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 21/50
28/28 [==============================] - 128s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 22/50
28/28 [==============================] - 129s 5s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 23/50
28/28 [==============================] -

Epoch 30/50
28/28 [==============================] - 14s 508ms/step - loss: 0.0643 - mse: 0.0643 - val_loss: 0.0594 - val_mse: 0.0594
Epoch 31/50
28/28 [==============================] - 13s 462ms/step - loss: 0.0645 - mse: 0.0645 - val_loss: 0.0651 - val_mse: 0.0651
Epoch 32/50
28/28 [==============================] - 13s 456ms/step - loss: 0.0675 - mse: 0.0675 - val_loss: 0.0494 - val_mse: 0.0494
Epoch 33/50
28/28 [==============================] - 13s 461ms/step - loss: 0.0701 - mse: 0.0701 - val_loss: 0.0533 - val_mse: 0.0533
Epoch 34/50
28/28 [==============================] - 13s 475ms/step - loss: 0.0758 - mse: 0.0758 - val_loss: 0.0509 - val_mse: 0.0509
Epoch 35/50
28/28 [==============================] - 14s 513ms/step - loss: 0.0778 - mse: 0.0778 - val_loss: 0.0747 - val_mse: 0.0747
Epoch 36/50
28/28 [==============================] - 13s 473ms/step - loss: 0.0786 - mse: 0.0786 - val_loss: 0.0771 - val_mse: 0.0771
Epoch 37/50
28/28 [==============================] - 15s 540ms

<a id='Part4_link'></a>
# 4. Visualize results

In [ ]:
# transform dictionary to dataframe
df_res = pd.DataFrame(res)

# store dataframe as csv locally
# df_res.to_csv('../data/02_results_run4.csv')

In [ ]:
# visualize results; use bubble plots to indicate magnitude of mean-square error for specific configuration comparing
# RNN to LSTM results

x_label = 'epochs'
y_label = 'units'
z_label = 'mse'
# condition_label = 'num_layers'
# condition_vals = list(set(df_res[condition_label]))

condition_LSTM = (df_res['model_type']=='LSTM')
condition_RNN = (df_res['model_type']=='RNN')

for model_type in ['LSTM', 'RNN']:
    condition_1 = (df_res['model_type']== model_type)
    
    x = df_res[condition_1][x_label].values
    y = df_res[condition_1][y_label].values
    
    z = df_res[condition_1][z_label].values
    plt.scatter(x, y, s=z*10000, alpha=0.6, c="red", linewidth=0.0)
        
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title('mean-square training error: {} model'.format(model_type))
    plt.show()

In [ ]:
df_res[df_res['model_type']=='RNN']